In [2]:
import simpy
import networkx as nx
from pulp import LpProblem, LpSolver, LpVariable, LpMinimize

# Define products and customer locations
products = ["pr1", "pr2"]
customer_locations = ["C1", "C2", "C3"]

# Randomly generate demand and production capacity (you can customize these)
demand = {
    "C1": {products[0]: 40, products[1]: 30},
    "C2": {products[0]: 25, products[1]: 15},
    "C3": {products[0]: 35, products[1]: 20},
}
production_capacity = {"P1": 100, "P2": 80}

# Define transportation cost matrix (you can customize this)
cost_matrix = {
    ("P1", "W1"): 5,
    ("P1", "W2"): 8,
    ("P2", "W1"): 7,
    ("P2", "W2"): 4,
    ("W1", "C1"): 3,
    ("W1", "C2"): 2,
    ("W1", "C3"): 6,
    ("W2", "C1"): 4,
    ("W2", "C2"): 1,
    ("W2", "C3"): 5,
}

# Initialize SimPy and NetworkX objects
env = simpy.Environment()
graph = nx.DiGraph()

# Define nodes and edges with initial inventory
for plant in ("P1", "P2"):
    graph.add_node(plant, inventory={p: 0 for p in products})
for warehouse in ("W1", "W2"):
    graph.add_node(warehouse, inventory={p: 0 for p in products})
for loc in customer_locations:
    graph.add_node(loc, demand=demand[loc])
for edge in cost_matrix:
    graph.add_edge(*edge)

# Define SimPy processes
class Production(simpy.Process):
    def __init__(self, env, plant, product, amount):
        super().__init__(env)
        self.plant = plant
        self.product = product
        self.amount = amount

    def run(self):
        yield env.timeout(5)  # Production time
        graph.nodes[self.plant]["inventory"][self.product] += self.amount

class Transportation(simpy.Process):
    def __init__(self, env, source, dest, product, amount):
        super().__init__(env)
        self.source = source
        self.dest = dest
        self.product = product
        self.amount = amount

    def run(self):
        yield env.timeout(cost_matrix[(self.source, self.dest)] / 2)  # Travel time
        graph.nodes[self.source]["inventory"][self.product] -= self.amount
        graph.nodes[self.dest]["inventory"][self.product] += self.amount

# Optimization function using PuLP
def optimize():
    problem = LpProblem("Supply Chain Optimization", LpMinimize)

    # Decision variables: amount of product transported from each source to each destination
    vars = {}
    for source in graph.nodes:
        for dest in graph.nodes:
            if source != dest:
                for product in products:
                    vars[(source, dest, product)] = LpVariable(0, production_capacity[source])

    # Objective function: minimize total transportation cost
    problem += sum(
        cost_matrix[(source, dest)] * vars[(source, dest, product)]
        for source in graph.nodes
        for dest in graph.nodes
        for product in products
    )

    # Demand constraints: demand at each customer location must be met
    for loc in customer_locations:
        for product in products:
            problem += sum(vars[(source, loc, product)] for source in graph.nodes) >= demand[loc][product]

        # Inventory constraints: inventory at each node cannot exceed capacity
    for node in graph.nodes:
        for product in products:
            problem += sum(
                vars[(source, node, product)] - vars[(node, dest, product)] for source, dest in graph.nodes
            ) <= production_capacity[node] if node in ("P1", "P2") else demand[node][product]

    # Solve the optimization problem
    solver = LpSolver(PULP_CBC_CMD)
    solver.solve(problem)

    # Update the simulation based on optimized decisions
    for source in graph.nodes:
        for dest in graph.nodes:
            for product in products:
                amount = solver.values(vars[(source, dest, product)])
                if amount > 0:
                    env.process(Transportation(env, source, dest, product, amount))

    # Start the simulation and stop when all demands are met
    running = True
    while running:
        running = False
        for loc in customer_locations:
            for product in products:
                if graph.nodes[loc]["inventory"][product] < demand[loc][product]:
                    running = True
                    break
        env.run(until=next(env.events) if running else None)

    # Visualization with NetworkX
    pos = nx.spring_layout(graph, k=2)
    nx.draw_networkx(graph, pos, with_labels=True)
    for edge in graph.edges:
        source, dest = edge
        for product, amount in graph.nodes[source]["inventory"].items():
            if amount > 0:
                label = f"{product}:{amount}"
                nx.draw_edge_labels(graph, pos, {(source, dest): label})
    
    plt.show()

# Run the simulation
optimize()




KeyError: 'W1'

In [ ]:
def move_product(env, source, dest, product, quantity):
    # Simulate movement of products from source to destination
    edge_cost = costs[(source, dest)][product]  # Retrieve transportation cost

    # If there's enough quantity to fulfill demand
    if quantity >= customers[dest][product]:
        quantity -= customers[dest][product]
        customers[dest][product] = 0
    else:
        customers[dest][product] -= quantity
        quantity = 0

    yield env.timeout(1)  # Simulating time taken for transportation

    G.edges[(source, dest, 0)]['quantity'] -= quantity  # Update the quantity on the edge
    print(f"Moving {quantity} units of {product} from {source} to {dest} with cost {edge_cost}")

    return quantity  # Return remaining quantity after transportation

def simulate_supply_chain(env, graph):
    while any(customers[customer][product] > 0 for customer in customers for product in customers[customer]):
        for edge in graph.edges(data=True):
            source, dest, data = edge[0], edge[1], edge[2]
            if data['quantity'] > 0:
                remaining_quantity = yield env.process(move_product(env, source, dest, data['product'], data['quantity']))
                graph.edges[(source, dest, 0)]['quantity'] = remaining_quantity

                if all(customers[customer][product] == 0 for customer in customers for product in customers[customer]):
                    return

# Start the simulation
env = simpy.Environment()
env.process(simulate_supply_chain(env, G))
env.run()

print("Simulation Complete")

# Visualize the final network state
plt.figure(figsize=(10, 6))
pos = nx.bipartite_layout(G, plants.keys())
nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', font_weight='bold', font_size=8)
edge_labels = {(u, v): f"{d['product']}:{d['quantity']}" for u, v, d in G.edges(data=True) if d['quantity'] > 0}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
plt.title('Final Supply Chain Network')
plt.show()
